In [ ]:
#from bs4 import BeautifulSoup
import re
import pandas as pd
import xml.etree.ElementTree as ET
from attribute import Attribute as att
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import *

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras import preprocessing
from keras_preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense, Dropout, Input, Embedding, Flatten, Dropout
from keras import Model
from sklearn.preprocessing import StandardScaler

In [ ]:
my_parser = ET.XMLParser(encoding='utf-8')
tree = ET.parse('dicom-2020a.dicomextract.txt', parser=my_parser)
#tree = ET.parse('xml DICOM', parser=my_parser)
root = tree.getroot()

In [ ]:
c=[]
c = root.findall(".//ItemEntry[@Type='1C']") + root.findall(".//ItemEntry[@Type='2C']")

In [ ]:
#Dictionary with conditional attributes:
attribute_dict = []
for a in c:
    description = a.attrib['Description']
    tag = a.attrib['Tag']
    type = a.attrib['Type']
    attribute_dicom = att(description,tag, type)
    attribute_dict.append(attribute_dicom)
attribute_original = deepcopy(attribute_dict)

In [ ]:
for a in attribute_original:
    print(a.description, '\n')
print(len(attribute_original))

In [ ]:
for a in attribute_dict:
    a.set_only_rule_def()
#
# for i in attribute_dict:
#     print(i.description, '\n')

In [ ]:
# Remove Notes, See Section, May be present
for a in attribute_dict:
    a.clean_note()
    a.clean_section()
    a.clean_may_be_present()
    a.clean_num_items_present()

for i in attribute_dict:
    print(i.description, '\n')


In [ ]:
for a in attribute_dict:
    a.clean_enumerated_values()

for i in attribute_dict:
    print(i.description, '\n')

In [ ]:
for a in attribute_dict:
    if 'be empty' in a.description:
        print(a.description)

In [ ]:
# ------------------------------- DATA PREPROCESSING ----------------------------------

In [ ]:
train_df = pd.read_csv('data.csv')

In [ ]:
cols = train_df.columns
label_cols = list(cols[0:7])
label_cols

In [ ]:
train_df['Attribute description']

In [ ]:
def clean_train_data(df, name):
    for i in range(len(df)):
        value = re.findall('[^\\(.*][A-Z]{2,}[^(.*\\)]', df[name][i])
        tag = re.findall('\\([A-Z\\d]{4},[A-Z\\d]{4}\\)', df[name][i])
        att = re.findall('(\\s\\b[A-Z]{1}[a-z]*\\b)', df[name][i])
        rep = re.findall(r'ATTRIBUTE*', df[name][i])
        if value:
            for val in value:
                df.loc[i,name]= df.loc[i,name].replace(val, " VALUE ")
        if tag:
            for t in tag:
                df.loc[i,name]= df.loc[i,name].replace(t, "TAG")

        if att:
            for a in range(len(att)):
                if a !=0:
                    df[name][i] = df[name][i].replace(att[a], "")
                else:
                    df[name][i] = df[name][i].replace(att[a], " ATTRIBUTE ")


    return df

In [ ]:
def clean_class_data(desc):
    for d in range(len(desc)):
        value = re.findall('[^\\(.*][A-Z]{2,}[^(.*\\)]', desc[d])
        tag = re.findall('\\([A-Z\\d]{4},[A-Z\\d]{4}\\)', desc[d])
        att = re.findall('(\\s\\b[A-Z]{1}[a-z]*\\b)', desc[d])
        rep = re.findall(r'ATTRIBUTE*', desc[d])
        if value:
            for val in value:
                desc[d] = desc[d].replace(val, " VALUE ")

        if tag:
            for t in tag:
                desc[d] = desc[d].replace(t, "TAG")

        if att:
            for a in range(len(att)):
                if a !=len(att):
                    desc[d] = desc[d].replace(att[a], "")
                else:
                    desc[d] = desc[d].replace(att[a], " ATTRIBUTE ")

    return desc

In [ ]:
# attribute_dict[10].description

In [ ]:
clean_train_data(train_df, 'Attribute description')
print(train_df.loc[0,'Attribute description'])
# print(cleaned_data.loc[0,'Attribute description'])
print(attribute_dict[0].description)

In [ ]:
p=train_df.columns
list(train_df.iloc[0,1:])

In [ ]:
# -----------------------------------------------MODEL TUNING AND TRAINING --------------------------------------------

In [ ]:
train_df['one_hot_labels'] = list(train_df[label_cols[1:7]].values)
labels = list(train_df['one_hot_labels'].values)

In [ ]:
desc = list(train_df['Attribute description'].values)
desc

In [ ]:
# labels

In [ ]:
X_train,X_test, Y_train, Y_test= train_test_split(desc,labels, test_size=0.1, random_state=100)

In [ ]:
tok=Tokenizer(
    num_words=90000,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789',
    lower=False,
    split=' ',
    char_level=False,
    oov_token=None,
    document_count=0
)
tok.fit_on_texts(X_train)
X_train=tok.texts_to_sequences(X_train)
tok.fit_on_texts(X_test)
X_test=tok.texts_to_sequences(X_test)

In [ ]:
maxlen = 500
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [ ]:
Y_test= np.asarray(Y_test)
Y_train= np.asarray(Y_train)

In [ ]:
X_train.shape, X_test.shape, Y_test.shape, Y_train.shape

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

In [ ]:
data=[]
for a in attribute_dict:
   data.append(a.description)
clean_class_data(data)
tok.fit_on_texts(data)
data_vector=tok.texts_to_sequences(data)

In [ ]:
data_vector = pad_sequences(data_vector, padding='post', maxlen=maxlen)
data_vector = sc.fit_transform(data_vector)

In [ ]:
svm = LinearSVC()
forest = RandomForestClassifier(random_state=40)

In [ ]:
cls = ['Attribute description', 'AttVal', 'AttValNot', 'AttPres',
       'ReqIfAttNotPres', 'NotReqIfAttPres', 'Txt']
multilabel_classifier = MultiOutputClassifier(estimator=svm)
# multilabel_classifier = MultiOutputClassifier(forest, n_jobs=2)
print(multilabel_classifier)

In [ ]:
multilabel_classifier = multilabel_classifier.fit(X_train, Y_train)

In [ ]:
# data_pred = pad_sequences(tok_data, padding='post', maxlen=maxlen)
yhat = multilabel_classifier.predict(X_test)
y_test_pred = multilabel_classifier.predict(data_vector)

In [ ]:
yhat[10]

In [ ]:
auc_y1 = roc_auc_score(Y_test[0],y_test_pred[0])
auc_y2 = roc_auc_score(Y_test[1],y_test_pred[1])

print("ROC AUC y1: %.4f, y2: %.4f" % (auc_y1, auc_y2))

In [ ]:
# sgd_linear_clf = SGDClassifier(loss='log', random_state=1, max_iter=5)
# clf_w = MultiOutputClassifier(sgd_linear_clf)
# clf_w.fit(X_train, Y_train)

In [ ]:
len(y_test_pred)

In [ ]:
for i in range(len(attribute_dict)):
    dsc = attribute_dict[i].description
    idx = y_test_pred[i]
    print('Description: ', dsc, '\n', 'Label: ', idx)

In [ ]:
data[len(data)-1]

In [ ]:
# ------------------------MODEL 2 ---------------------------------

In [ ]:
input_1 = Input(shape=(maxlen,))
embedding_layer = Embedding(10000, 128, trainable=True)(input_1)

LSTM_Layer1 = (LSTM(128, return_sequences=True))(embedding_layer)
dropout_1 = Dropout(0.3)(LSTM_Layer1)

LSTM_Layer2 = (LSTM(128))(dropout_1)
dropout_2 = Dropout(0.3)(LSTM_Layer2)
dense_2 = Flatten()(dropout_2)

output1 = Dense(6, activation='softmax')(dense_2)  #fake news
#output2= Dense(8,activation='softmax')(dense_2)     #news subject

model = Model(inputs=input_1, outputs=output1)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()
model.fit(x=X_train, y=Y_train, batch_size=32, epochs=50)

In [ ]:
model.summary()

In [ ]:
model.fit(x=X_train,y=Y_train, batch_size=32, epochs=100)

In [ ]:
loss,acc=model.evaluate(x=X_test,y=Y_test)

In [ ]:
data = []
for a in attribute_dict:
    data.append(a.description)
tok.fit_on_texts(data)
tok_data = tok.texts_to_sequences(data)

tok_data = pad_sequences(tok_data, padding='post', maxlen=maxlen)
tok_data

In [ ]:
y_val_predicted_labels = model.predict(data_vector)
len(y_val_predicted_labels)

In [ ]:
(y_val_predicted_labels[4])

In [ ]:
for i in range(len(attribute_dict)):
    dsc = attribute_dict[i].description
    idx = np.argmax(y_val_predicted_labels[i])
    print('Description: ', dsc, '\n', 'Label: ', labels[idx])

In [1]:
import DataRead as dr

In [2]:
reader = dr.LabelledData()
dataset = list(reader.read("data.csv"))

print("type of its first element: ", dataset[9])
print("size of dataset: ", len(dataset))

type of its first element:  Instance with fields:
 	 text: TextField of length 9 with text: 
 		[Required, if, VALUE, ATTRIBUTE, TAG, value, is, VALUE, .]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 labels: MultiLabelField with labels: ['AttVal', ' ', ' '] in namespace: 'labels'.' 

size of dataset:  108
